In [24]:
#@title
import numpy as np
import math
import cmath
from functools import reduce
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
#@title
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]], dtype = 'complex_')
X = np.array([[0,1],[1,0]], dtype = 'complex_')
I = np.array([[1,0],[0,1]], dtype = 'complex_')
Z = np.array([[1,0],[0,-1]], dtype = 'complex_')
T = np.array([[1,0],[0, cmath.exp((1j*cmath.pi)/4)]], dtype = 'complex_')
T_ermit = np.array([[1,0],[0, cmath.exp((-1j*cmath.pi)/4)]], dtype = 'complex_')
V = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1+1j)/2,(1-1j)/2],[0,0,(1-1j)/2,(1+1j)/2]], dtype = 'complex_')
V_ermit = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1-1j)/2,(1+1j)/2],[0,0,(1+1j)/2,(1-1j)/2]], dtype = 'complex_')
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype = 'complex_')
Zgate = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]], dtype = 'complex_')
Toffoli = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 1, 0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 0, 0, 0],
           [0, 0, 0, 0, 0, 1, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 1],
           [0, 0, 0, 0, 0, 0, 1, 0]],dtype = 'complex_')




def UnitGate(q, matrix, q0):
  qubit = deepcopy(q)
  q0 = q0-1
  for i in range(len(qubit)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r = 0
    else:
      r = 1
    qubit[i] = q[i|(1<<q0)] * matrix[r][1] + q[(i|(1<<q0))^(1<<q0)] * matrix[r][0]
  return qubit

#if you need to impose some kind of control operation NOT ,
# then you need to write (-) and the desired number. 

def UnitGatetwo(q,matrix,q1,q0):
  qubit = deepcopy(q)
  if q1 > 0:
    q0 = q0-1
    q1 = q1-1
    for i in range(len(q)):
      if (i % (1<<(q0+1))) < (1<<(q0)):
        r1 = 0
      else:
        r1 = 1
      if (i %(1<<(q1+1))) < (1<<(q1)):
        r2 = 0
      else:
        r2 = 1
      z = 2*r2 +r1
      y = (i|(1<<q0))|(1<<q1)
      qubit[i] = q[y] * matrix[z][3]  \
      + q[(y)^(1<<q0)] * matrix[z][2] \
      + q[(y)^(1<<q1)] * matrix[z][1] \
      + q[((y)^(1<<q1))^(1<<q0)] * matrix[z][0]
  elif q1 <0:
    q1 = -q1
    qubit = UnitGate(qubit,X,q1)
    qubit = UnitGatetwo(qubit,matrix,q1,q0)
    qubit = UnitGate(qubit,X,q1)
  else:
    print('break q1 = 0')

  return qubit

def UnitGatethree(qubit, matrix, q2, q1, q0):
    q = deepcopy(qubit)
    if q2 > 0 and q1 > 0:
      q0 = q0 - 1
      q2 = q2 - 1
      q1 = q1 - 1
      for i in range(len(q)):
          if (i % (1 << (q0 + 1))) < (1 << q0):
              r1 = 0
          else:
              r1 = 1
          if (i % (1 << (q1 + 1))) < (1 << q1):
              r2 = 0
          else:
              r2 = 1
          if (i % (1 << (q2 + 1))) < (1 << q2):
              r3 = 0
          else:
              r3 = 1
          z = 4*r3+2*r2+r1
          y =(i|(1<<q0))|(1<<q1)|(1<<q2)
          q[i] = qubit[y]*matrix[z][7] \
          + qubit[(y)^(1<<q0)]*matrix[z][6] \
          + qubit[(y)^(1<<q1)]*matrix[z][5] \
          + qubit[(y)^(1<<q0)^(1<<q1)]*matrix[z][4] \
          + qubit[(y)^(1<<q2)]*matrix[z][3] \
          + qubit[(y)^(1<<q0)^(1<<q2)]*matrix[z][2] \
          + qubit[(y)^(1<<q1)^(1<<q2)]*matrix[z][1] \
          + qubit[(y)^(1<<q1)^(1<<q2)^(1<<q0)]*matrix[z][0]
    elif q2 < 0:
      q2 = -q2
      q = UnitGate(q,X,q2)
      q = UnitGatethree(q,matrix, q2,q1,q0)
      q = UnitGate(q,X,q2)
    elif q1 < 0:
      q1 = -q1
      q = UnitGate(q,X,q1)
      q = UnitGatethree(q,matrix, q2, q1,q0)
      q = UnitGate(q,X,q1)
    else:
      print('break q1 = 0')

    return q

# Translation of the i qubit by count into a bit record of size n
# Input (n) is the number of qubits, (i) is the number of qubits 
# int output bit entry
def inttobit(n, i):
  return list(map(int, list(format(i, "0{0}b".format(n)))))

def powcomplex(complexmatrix):
  return (complexmatrix * (complexmatrix.real - 1j*complexmatrix.imag)).real

In [3]:
#@title
#qubit - the entire qubit vector
#maxdop- the maximum number of additional qubits required to implement an n-gate operation
##nstart - the number of qubits without additional
#N - how many qubit operation, for toffoli = 3, for CNOT = 2
#matrix - controlled matrix
#*args - all arguments that are affected, the last controlled
def CnNOT(qubit,maxdop,nstart,N,matrix,*args):
  qubit = UnitGatethree(qubit,Toffoli,args[0],args[1],nstart+1)
  for i in range(N-3):
    qubit = UnitGatethree(qubit,Toffoli,args[2+i],nstart+1+i,nstart+i+2)
  qubit = UnitGatetwo(qubit,matrix,nstart+maxdop,args[-1])
  for i in range((N-2),1,-1):
    qubit = UnitGatethree(qubit,Toffoli,args[i],nstart+i-1,nstart+i)
  qubit = UnitGatethree(qubit,Toffoli,args[0],args[1],nstart+1)
  return qubit  

**Grover's algorithm**

In [4]:
#@title
#qubit - начальные кубиты
#iter - число итераций
#n - начальное количество кубит
#step - количество шагов
#*args - несколько списков вида(1, 2, -3), где нет последнего контролируемого кубита
def oraclsatproblem(qubit,n,step, *args):
  for i in range(step):
    if len(args[i]) == 1:
      qubit = UnitGatetwo(qubit,CNOT,args[i][0], n+i+1)
    elif len(args[i]) == 2:
      qubit = UnitGatethree(qubit,Toffoli, args[i][0], args[i][1], n+i+1)
    else:
      qubit = CnNOT(qubit,len(args[i])-1,n+1+step,len(args[i])+1,CNOT,*args[i],n+i+1)

  if step == 1:
    qubit = UnitGatetwo(qubit,CNOT,-(n+1),n+step+1)
  elif step == 2:
    qubit = UnitGatethree(qubit,Toffoli,-(n+1),-(n+2),n+step+1)
  else:
    qubit = CnNOT(qubit,step-1,n+step+1,step+1,CNOT,*[-(i+1) for i in range(n,n+step)],n+step+1)
  for i in range(step-1,-1,-1):
    if len(args[i]) == 1:
      qubit = UnitGatetwo(qubit,CNOT,args[i][0], n+i+1)
    elif len(args[i]) == 2:
      qubit = UnitGatethree(qubit,Toffoli, args[i][0], args[i][1], n+i+1)
    else:
      qubit = CnNOT(qubit,len(args[i])-1,n+1+step,len(args[i])+1,CNOT,*args[i],n+i+1)
  return qubit

def musor(qubit,n,step, *args):
  for i in range(step):
    if len(args[i]) == 1:
      qubit = UnitGatetwo(qubit,CNOT,args[i][0], n+i+1)
    elif len(args[i]) == 2:
      qubit = UnitGatethree(qubit,Toffoli, args[i][0], args[i][1], n+i+1)
    else:
      qubit = CnNOT(qubit,len(args[i])-1,n+1+step,len(args[i])+1,CNOT,*args[i],n+i+1)

  if step == 1:
    qubit = UnitGatetwo(qubit,CNOT,-(n+1),n+step+1)
  elif step == 2:
    qubit = UnitGatethree(qubit,Toffoli,-(n+1),-(n+2),n+step+1)
  else:
    qubit = CnNOT(qubit,step-1,n+step+1,step+1,CNOT,*[-(i+1) for i in range(n,n+step)],n+step+1)
  return qubit

def diffuzer(qubit,n,contrqubit):
  if n == 2:
    qubit = UnitGate(qubit,H,1)
    qubit = UnitGate(qubit,H,2)
    qubit = UnitGatethree(qubit,Toffoli,-1,-2,contrqubit)
    qubit = UnitGate(qubit,H,1)
    qubit = UnitGate(qubit,H,2)
  if n >= 3:
    for j in range(n):
      qubit = UnitGate(qubit,H,j+1)
    qubit = CnNOT(qubit,n-2,n+step+1,n+1,CNOT,*[-(i+1) for i in range(n)],contrqubit)
    for j in range(n):
      qubit = UnitGate(qubit,H,j+1)
  return qubit

def Groversonecirc(q,iter,n,step, *args):
  q = UnitGate(q,X,n+step+1)
  q = UnitGate(q,H,n+step+1)
  for i in range(n):
    q = UnitGate(q,H,i+1)
  for i in range(iter):
    q = oraclsatproblem(q,n,step, *args)
    q = diffuzer(q,n,n+step+1)
  q = musor(q,n,step, *args)
  return q

The SAT problem

n is the number of initial qubits (the number of variables in the sat problem)

step - number of steps 

maxstep is the largest gate (the number of qubits participating in this gate)

Set the steps in the code itself!

P.s since the number of answers to the solutions is unknown in the sat problem, an additional qubit is introduced to consider the limiting case at M< (2^n)/2, therefore, when calculating the maximum step, it should also be taken into account

Example 1:

3 variables : x1 x2 x3

(x1 or x2) and (not x2)

One iteration of oracle + diffuser

https://algassert.com/quirk#circuit={%22cols%22:[[1,1,1,1,1,1,%22X%22],[1,1,1,1,1,1,%22H%22],[%22Chance3%22],[%22H%22,%22H%22,%22H%22,%22H%22],[1,1,1,1,1,1,%22Bloch%22],[%22%E2%97%A6%22,%22%E2%97%A6%22,1,1,%22X%22],[%22%E2%80%A2%22,1,1,1,1,%22X%22],[1,1,1,1,%22%E2%97%A6%22,%22%E2%97%A6%22,%22X%22],[1,%22%E2%80%A2%22,1,1,1,%22X%22],[%22%E2%97%A6%22,%22%E2%97%A6%22,1,1,%22X%22],[%22Chance%22,%22Chance%22,%22Chance%22,%22Chance%22,%22Chance%22,%22Chance%22],[%22H%22,%22H%22,%22H%22,%22H%22],[%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,1,1,%22X%22],[%22H%22,%22H%22,%22H%22,%22H%22],[%22Chance3%22,1,1,1,1,1,%22Bloch%22]]}

n = 3

step = 2

maxstep = 5

Should be : 100 and 101

In [85]:
n = int(input())
n=n+1
step = int(input())
maxstep = int(input())
iter = round(math.pi /4 * math.sqrt((2*2**n)))
q = np.zeros((2**(n+step+maxstep-1)), dtype = 'complex_')
q[0] = 1
l1 = [-1,-2]
l2 = [2]
q = Groversonecirc(q,iter,n,step, l1,l2)
pow1 = powcomplex(q)
pow2 = np.where(pow1 > 0.1)
pow3 = pow2[0]
for i in range(len(pow3)):
  print((inttobit(n+step+1+maxstep-2,pow3[i]))[:maxstep-2+step+1:-1])

3
2
5
[1, 0, 0]
[1, 0, 1]
[1, 0, 0]
[1, 0, 1]
[1, 0, 0]
[1, 0, 1]
[1, 0, 0]
[1, 0, 1]


In [84]:
pow2 = np.where(pow1 > 0.01)
pow3 = pow2[0]
a = []
for i in range(len(pow3)):
  a.append((inttobit(n+step+1+maxstep-2,pow3[i]))[:maxstep-2+step+1:-1])
numbers = a

def get_unique_numbers(numbers):
    unique = []
    for number in numbers:
        if number not in unique:
            unique.append(number)
    return unique
get_unique_numbers(numbers)

[[1, 0, 0], [1, 0, 1]]

Example 2

((((not x1) or (not x3) or (not x4)) and (x2 or x3 or (not x4)) and (x1 or (not x2) or x4) and ((not x1) or x2 or (not x3))

One iteration: Oracle + Diffuser

https://algassert.com/quirk#circuit={%22cols%22:[[%22H%22,%22H%22,%22H%22,%22H%22,%22H%22,1,1,1,1,%22X%22],[1,1,1,1,1,1,1,1,1,%22H%22],[%22%E2%80%A2%22,1,%22%E2%80%A2%22,%22%E2%80%A2%22,1,%22X%22],[1,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%80%A2%22,1,1,%22X%22],[%22%E2%97%A6%22,%22%E2%80%A2%22,1,%22%E2%97%A6%22,1,1,1,%22X%22],[%22%E2%80%A2%22,%22%E2%97%A6%22,%22%E2%80%A2%22,1,1,1,1,1,%22X%22],[1,1,1,1,1,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,%22X%22],[%22Chance%22,%22Chance%22,%22Chance%22,%22Chance%22,%22Chance%22,1,1,1,1,%22Chance%22],[%22%E2%80%A2%22,%22%E2%97%A6%22,%22%E2%80%A2%22,1,1,1,1,1,%22X%22],[%22%E2%97%A6%22,%22%E2%80%A2%22,1,%22%E2%97%A6%22,1,1,1,%22X%22],[1,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%80%A2%22,1,1,%22X%22],[%22%E2%80%A2%22,1,%22%E2%80%A2%22,%22%E2%80%A2%22,1,%22X%22],[%22H%22,%22H%22,%22H%22,%22H%22,%22H%22],[%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,%22%E2%97%A6%22,1,1,1,1,%22X%22],[%22H%22,%22H%22,%22H%22,%22H%22,%22H%22]]}

n = 4 

step = 4

 maxstep = 6

Must be :

0000; 0010; 0011; 0101; 0111; 1000; 1010; 1100; 1101; 1110

In [86]:
n = int(input())
n = n+1
step = int(input())
maxstep = int(input())
iter = round(math.pi /4 * math.sqrt((2*2**n)))
q = np.zeros((2**(n+step+maxstep-1)), dtype = 'complex_')
q[0] = 1
l1 = [1,3,4]
l2 = [-2,-3,4]
l3 = [-1,2,-4]
l4 = [1,-2,3]
q = Groversonecirc(q,iter,n,step, l1,l2,l3,l4)
pow1 = powcomplex(q)
pow2 = np.where(pow1 > 0.03)
pow3 = pow2[0]
for i in range(len(pow3)):
  print((inttobit(n+step+1+maxstep-2,pow3[i]))[:maxstep-2+step+1:-1])

4
4
6


In [87]:
pow2 = np.where(pow1 > 0.005)
pow3 = pow2[0]
a = []
for i in range(len(pow3)):
  a.append((inttobit(n+step+1+maxstep-2,pow3[i]))[:maxstep-2+step+1:-1])
numbers = a

def get_unique_numbers(numbers):
    unique = []
    for number in numbers:
        if number not in unique:
            unique.append(number)
    return unique
get_unique_numbers(numbers)

[[0, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 1, 1, 0],
 [0, 1, 0, 1],
 [1, 1, 0, 1],
 [0, 0, 1, 1],
 [0, 1, 1, 1]]

Checking 1 and 2 examples

In [ ]:
for x1 in range(2):
  for x2 in range(2):
    for x3 in range(2):
      if ((x1 or x2) and (not x2)) == 1:
        print(x1,x2,x3)

1 0 0
1 0 1


In [ ]:
for x1 in range(2):
  for x2 in range(2):
    for x3 in range(2):
      for x4 in range(2):
        if (((not x1) or (not x3) or (not x4)) and (x2 or x3 or (not x4)) and (x1 or (not x2) or x4) and ((not x1) or x2 or (not x3))) == 1:
          print(x1,x2,x3,x4)

0 0 0 0
0 0 1 0
0 0 1 1
0 1 0 1
0 1 1 1
1 0 0 0
1 1 0 0
1 1 0 1
1 1 1 0
